In [1]:
from bs4 import BeautifulSoup as bs
import requests
import pandas as pd
import re

In [2]:
date="2017.03.28"
url_part1="""http://dart.fss.or.kr/dsac001/mainAll.do?selectDate="""
url_part2="""&sort=&series=&mdayCnt=0&currentPage="""

url=url_part1+date+url_part2
print(url)

http://dart.fss.or.kr/dsac001/mainAll.do?selectDate=2017.03.28&sort=&series=&mdayCnt=0&currentPage=


In [4]:
res=requests.get(url) #urllib.request.urlopen() 과 동일한 역할
soup=bs(res.text,'html.parser')

In [8]:
# 총 몇 건의 데이터 있는지 확인

In [9]:
for i in range(0,len(soup.find_all("p"))):
    print(i)
    print(soup.find_all("p")[i])
    print("---------------------")

0
<p id="accessibility"><a href="#layoutMain">본문 바로가기</a></p>
---------------------
1
<p class="utill">
<!-- <ul class="topLink"> -->
<a href="/dsag002/loginForm.do"><img alt="로그인" src="/images/common/utill_login.gif" title="로그인"/></a>
<a href="/dsag003/main.do"><img alt="마이페이지" src="/images/common/utill_mypage.gif" title="마이페이지"/></a>
<a href="/dsaa003/searchGuide.do"><img alt="공시업무" src="/images/common/utill_gongsi.gif" title="공시업무"/></a>
<a href="/introduction/content1.do"><img alt="다트소개" src="/images/common/utill_intro.gif" title="다트소개"/></a>
<a href="https://opendart.fss.or.kr/intro/main.do" target="_blank"><img alt="오픈API" src="/images/common/utill_openapi.gif" title="오픈API"/></a>
<a href="/introduction/content6.do"><img alt="RSS이용안내" src="/images/common/utill_rss.gif" title="RSS이용안내"/></a>
<a class="end" href="/sitemap.do"><img alt="사이트맵" src="/images/common/utill_sitemap.gif" title="사이트맵"/></a>
</p>
---------------------
2
<p class="tit"><img alt="최근공시" src="/images/common/lnb_

In [74]:
totNum=soup.find_all("p")[8].text

In [44]:
a="                aaaaaaaaaaaaaaa           bbbbbbbbbbbbbb          cccccccccccccccc        "
print(a.strip())   # 앞 뒤만 공란 제거
print(a.replace(" ",""))    # 모든 공란 제거

aaaaaaaaaaaaaaa           bbbbbbbbbbbbbb          cccccccccccccccc
aaaaaaaaaaaaaaabbbbbbbbbbbbbbcccccccccccccccc


In [75]:
# 정규식 활용해서 갯수 꺼내오기
"""
re.sub(찾을 패턴, 찾고나서 변경할 값, 어디서 할지 대상)
"""
re.sub(r"\[총","",totNum) # "[총"이란 값을 찾아서 ""로 변경함 

'\r\n\t\t[1/9]  894건]\r\n\t'

In [76]:
totNum=re.sub(r"\[.*\] \[총", "",totNum)  # '['로 시작하고 ']'' 끝나고 '[총'로 시작하는 것 찾아서 삭제
totNum=re.sub(r"건]","",totNum)
totNum=totNum.strip()
totNum=int(totNum.strip())

# 몇페이지 표시해야 하는지 찾기
totPage=(totNum//100)+1
totPage

9

In [91]:
# 1페이지에는 몇건의 공시 정보가 있는지 찾아보세요
# 주요 태그는 tr
# 갯수 > len


In [178]:
"""
1. 시장종류(코넥스, 코스닥) : Cat
2. 회사이름 : comName
3. 회사ID : coID
4. 공시 내용/제목 : Content
5. 해당 공시 고유값 : rcpNo
6. 공시 제출 시간 : pubTime
7. 공시 날짜 : pubDate
8. 공시 요청 날짜 : reqDate
"""
print(soup.find_all("tr")[2].find_all("td")[1].find("img").get("title"))  #시장 종류, 태그 안에 값 가져올 때 (get)
print(soup.find_all("tr")[2].find("a").text.strip()) # 회사이름 
print(re.findall(r"[0-9]{8}",soup.find_all("tr")[2].find("a").get("href"))[0]) # 회사ID ,연속된 숫자 8자리 정규식으로 표현, [0-9]{8}
print(re.sub("\\t","",re.sub("\\n","",re.sub("\\r","",soup.find_all("tr")[2].find_all("td")[2].text.replace(" ",""))))) # 공시제목
#print(soup.find_all("tr")[2].find_all("td")[2].text.replace(" ","").replace("\n","").replace("\t","").replace("\r",""))
print(re.findall(r"[0-9]{14}",soup.find_all("tr")[2].find_all("td")[2].find("a").get("href"))[0]) # 공시 고유값
print(soup.find_all("tr")[2].find("td").text.strip()) #공시 제출 시간
print(soup.find_all("tr")[2].find_all("td",class_="cen_txt")[1].text) # 공시 날짜
print(soup.find_all("tr")[2].find_all("td",class_="cen_txt")[1].text) # 공시 요청 날짜




코스닥시장
쏠리드
00364403
불성실공시법인지정예고(공시불이행)
20170328901048
18:46
2017.03.28


In [6]:
from bs4 import BeautifulSoup as bs
import requests
import pandas as pd
import re

#date=input("날짜를 입력하세요: (oooo.oo.oo 식으로)")
date="2017.03.28"
url_part1="""http://dart.fss.or.kr/dsac001/mainAll.do?selectDate="""
url_part2="""&sort=&series=&mdayCnt=0&currentPage="""

url=url_part1+date+url_part2

res=requests.get(url) #urllib.request.urlopen() 과 동일한 역할
soup=bs(res.text,'html.parser')

dart = pd.DataFrame()
dart_temp = pd.DataFrame()
dart_temp = dart_temp.append({"Cat":"","comName":"","coID":"","Content":"","rcpNo":"","pubTime":"","pubDate":"","reqDate":""},ignore_index=True)

totNum=soup.find_all("p")[8].text
totNum=re.sub(r"\[.*\] \[총", "",totNum)  # '['로 시작하고 ']'' 끝나고 '[총'로 시작하는 것 찾아서 삭제
totNum=re.sub(r"건]","",totNum)
totNum=totNum.strip()
totNum=int(totNum.strip())

totPage=(totNum//100)+1

num=len(soup.find_all("tr"))


#############################################

for k in range(1,totPage+1):
    date
    url_part1="""http://dart.fss.or.kr/dsac001/mainAll.do?selectDate="""
    url_part2="""&sort=&series=&mdayCnt=0&currentPage="""+str(k)

    url=url_part1+date+url_part2
    res=requests.get(url) #urllib.request.urlopen() 과 동일한 역할
    soup=bs(res.text,'html.parser')
    
    num=len(soup.find_all("tr"))
    for i in range(1,num):
#         print(soup.find_all("tr")[100])
        dart_temp.at[0,"Cat"]=soup.find_all("tr")[i].find_all("td")[1].find("img").get("title")    
        dart_temp.at[0,"comName"]=soup.find_all("tr")[i].find("a").text.strip()
        dart_temp.at[0,"coID"]=re.findall(r"[0-9]{8}",soup.find_all("tr")[i].find("a").get("href"))[0]
        dart_temp.at[0,"Content"]=re.sub("\\t","",re.sub("\\n","",re.sub("\\r","",soup.find_all("tr")[i].find_all("td")[2].text.replace(" ",""))))
        dart_temp.at[0,"rcpNo"]=re.findall(r"[0-9]{14}",soup.find_all("tr")[i].find_all("td")[2].find("a").get("href"))[0]
        dart_temp.at[0,"pubTime"]=soup.find_all("tr")[i].find("td").text.strip()
        dart_temp.at[0,"pubDate"]=date
        dart_temp.at[0,"reqDate"]=soup.find_all("tr")[i].find_all("td",class_="cen_txt")[1].text
        
#         if movieData["movieListResult"]["movieList"][i]["directors"] !=[]:
#             movieDOF_temp.iat[0,temp_movdir_idx]=movieData["movieListResult"]["movieList"][i]["directors"][0]["peopleNm"]
#         else:
#             pass
    
        dart=dart.append(dart_temp, ignore_index=True)
    
        for j in range(0,len(dart_temp.columns)):
            dart_temp.iat[0,j]=""
dart

,Cat,Content,coID,comName,pubDate,pubTime,rcpNo,reqDate
0,코넥스시장,감사보고서제출,01109690,도부마스크,2017.03.28,18:58,20170328601049,2017.03.28
1,코스닥시장,불성실공시법인지정예고(공시불이행),00364403,쏠리드,2017.03.28,18:46,20170328901048,2017.03.28
2,코스닥시장,주권매매거래정지(불성실공시법인지정),00308559,코디,2017.03.28,18:35,20170328901037,2017.03.28
3,코스닥시장,불성실공시법인지정(공시불이행),00308559,코디,2017.03.28,18:35,20170328901047,2017.03.28
4,기타법인,주권매매거래정지(불성실공시법인지정),00207223,위너지스,2017.03.28,18:29,20170328901039,2017.03.28
...,...,...,...,...,...,...,...,...
889,기타법인,임원의변동,00178806,바이더웨이,2017.03.28,07:00,20170327000959,2017.03.28
890,기타법인,[기재정정]감사보고서(2016.12),01083000,케이엔에프인터내셔날,2017.03.28,07:00,20170327000967,2017.03.28
891,기타법인,감사보고서(2016.12),01078868,케이피엑스글로벌,2017.03.28,07:00,20170327000958,2017.03.28
892,기타법인,동일인등출자계열회사와의상품ㆍ용역거래,00186513,코리아세븐,2017.03.28,07:00,20170327000962,2017.03.28


In [92]:
from bs4 import BeautifulSoup as bs
import requests
import pandas as pd
import re
from datetime import datetime, timedelta

dart = pd.DataFrame()
dart_temp = pd.DataFrame()
dart_temp = dart_temp.append({"Cat":"","comName":"","coID":"","Content":"","rcpNo":"","pubTime":"","pubDate":"","reqDate":""},ignore_index=True)

date1=input("[같은 월만 됩니다.]년을 입력하세요ㅛㅛㅛㅛㅛ")
date2=input("월을 입력하세요ㅛㅛㅛㅛㅛㅛㅛㅛ")
date3=input("시작일을 입력하세요ㅛㅛㅛㅛㅛㅛㅛㅛ")
date4=input("종료일을 입력하세요ㅛㅛㅛㅛㅛㅛㅛㅛ")
date2=date2.zfill(2)
date3=date3.zfill(2)
date4=date4.zfill(2)

if ((int(date1)%4==0)and(int(date1)%100!=0))or(int(date1)%400==0):
    a1=[31,29,31,30,31,30,31,31,30,31,30,31]
else:
    a1=[31,28,31,30,31,30,31,31,30,31,30,31]
  


date_start=datetime(int(date1),int(date2),int(date3))
date_end=datetime(int(date1),int(date2),int(date4))
day1=date_end-date_start


for a in range(0,day1.days+1):
#     if((int(date3)+a)):
        
    date=date1+date2+str(int(date3)+a).zfill(2)
    
    print(date)
    
    url_part1="""http://dart.fss.or.kr/dsac001/mainAll.do?selectDate="""
    url_part2="""&sort=&series=&mdayCnt=0&currentPage="""

    url=url_part1+date+url_part2

    res=requests.get(url) #urllib.request.urlopen() 과 동일한 역할
    soup=bs(res.text,'html.parser')

    
   
    if(soup.find_all("p")[8].text.strip()=="상기 보고서명 앞의 대괄호([]) 및 비고란의 약어(예:)에 마우스를 위치하면 설명이 나타나니 참고하시기 바랍니다."):
        continue
    else:
        totNum=soup.find_all("p")[8].text
        
    
    totNum=re.sub(r"\[.*\] \[총", "",totNum)  # '['로 시작하고 ']'' 끝나고 '[총'로 시작하는 것 찾아서 삭제
    totNum=re.sub(r"건]","",totNum)
    totNum=totNum.strip()
   
    totNum=int(totNum.strip())

    totPage=(totNum//100)+1

    num=len(soup.find_all("tr"))


    #############################################

    for k in range(1,totPage+1):
        date
        url_part1="""http://dart.fss.or.kr/dsac001/mainAll.do?selectDate="""
        url_part2="""&sort=&series=&mdayCnt=0&currentPage="""+str(k)

        url=url_part1+date+url_part2
        res=requests.get(url) #urllib.request.urlopen() 과 동일한 역할
        soup=bs(res.text,'html.parser')
    
        num=len(soup.find_all("tr"))
        for i in range(1,num):
    #         print(soup.find_all("tr")[100])
            
            if len(soup.find_all("tr")[i].find_all("td"))>=2:
                dart_temp.at[0,"Cat"]=soup.find_all("tr")[i].find_all("td")[1].find("img").get("title") 
            #dart_temp.at[0,"Cat"]=soup.find_all("tr")[i].find_all("td")[1].find("img").get("title")    
                dart_temp.at[0,"comName"]=soup.find_all("tr")[i].find("a").text.strip()
                dart_temp.at[0,"coID"]=re.findall(r"[0-9]{8}",soup.find_all("tr")[i].find("a").get("href"))[0]
                dart_temp.at[0,"Content"]=re.sub("\\t","",re.sub("\\n","",re.sub("\\r","",soup.find_all("tr")[i].find_all("td")[2].text.replace(" ",""))))
                dart_temp.at[0,"rcpNo"]=re.findall(r"[0-9]{14}",soup.find_all("tr")[i].find_all("td")[2].find("a").get("href"))[0]
                dart_temp.at[0,"pubTime"]=soup.find_all("tr")[i].find("td").text.strip()
                dart_temp.at[0,"pubDate"]=date
                dart_temp.at[0,"reqDate"]=soup.find_all("tr")[i].find_all("td",class_="cen_txt")[1].text

    #         if movieData["movieListResult"]["movieList"][i]["directors"] !=[]:
    #             movieDOF_temp.iat[0,temp_movdir_idx]=movieData["movieListResult"]["movieList"][i]["directors"][0]["peopleNm"]
    #         else:
    #             pass

            dart=dart.append(dart_temp, ignore_index=True)

            for j in range(0,len(dart_temp.columns)):
                dart_temp.iat[0,j]=""
dart

[같은 월만 됩니다.]년을 입력하세요ㅛㅛㅛㅛㅛ2017
월을 입력하세요ㅛㅛㅛㅛㅛㅛㅛㅛ3
시작일을 입력하세요ㅛㅛㅛㅛㅛㅛㅛㅛ1
종료일을 입력하세요ㅛㅛㅛㅛㅛㅛㅛㅛ3
20170301
20170302
20170303


,Cat,Content,coID,comName,pubDate,pubTime,rcpNo,reqDate
0,코스닥시장,타법인주식및출자증권취득결정(자율공시),00226228,한프,20170302,20:46,20170302901111,2017.03.02
1,코스닥시장,주권관련사채권의취득결정,00226228,한프,20170302,20:45,20170302901133,2017.03.02
2,코스닥시장,기타시장안내(상장적격성실질심사관련),00188715,현진소재,20170302,19:01,20170302901109,2017.03.02
3,코스닥시장,주권매매거래정지(투자자보호),00188715,현진소재,20170302,19:01,20170302901128,2017.03.02
4,코스닥시장,최대주주변경을수반하는주식양수도계약체결,00188715,현진소재,20170302,18:55,20170302901127,2017.03.02
...,...,...,...,...,...,...,...,...
1647,기타법인,특수관계인으로부터자금차입,00692193,부영CC,20170303,07:00,20170302000592,2017.03.03
1648,기타법인,주주총회소집공고,00159005,생고뱅코리아홀딩스,20170303,07:00,20170302000598,2017.03.03
1649,기타법인,주요사항보고서(소송등의제기),00527880,썬테크놀로지스,20170303,07:00,20170302000595,2017.03.03
1650,코스닥시장,[기재정정]감사보고서(2015.12),00962223,지니언스,20170303,07:00,20170302000591,2017.03.03
